# Imports

In [1]:
from langchain_ollama import OllamaLLM # Para levantar el modelo LLama3.1
from langchain_community.document_loaders import PyPDFDirectoryLoader #Para Cargar los PDFs
from langchain.text_splitter import RecursiveCharacterTextSplitter # Para hacer los CHUNKS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings # Para hacer el embeddings de los CHUNKS
from fastembed import TextEmbedding
from langchain_chroma import Chroma# Para trabajar con la BD vectorial
from langchain.prompts import PromptTemplate # Para hacer el template del prompt del modelo LLama
from langchain.chains import RetrievalQA # Para hacer las preguntas al modelo y obtener las respuestas

c:\Users\claud\Desktop\Cursos\LearningLangChain\thisEnvironment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import re

# Parámetros

In [3]:
CHUNK_SIZE=2000
CHUNK_OVERLAP=500

# Consideraciones
* Se tomó la decisión de que todos los documentos van en una sola colección

# Modelo
* El modelo es el llama3.1
* Debe de estar installado el modelo y ollama.
* [Descargar Ollama](https://ollama.com/)
* Descargar el modelo en la terminal luego de descargar OLLAMA `ollama pull llama3.1` **Pesa 4GB**


In [4]:
llm = OllamaLLM(model="llama3.1")
llm.invoke("Hola, quien eres?")

'Hola! Soy un modelo de lenguaje artificial creado por Meta, y no tengo una identidad personal en el sentido tradicional. No tengo un nombre, edad, género ni ubicación específica. Mi función es ayudar a las personas a encontrar la información que necesitan, responder preguntas y facilitar la comunicación en varios idiomas.\n\nPuedes llamarme "asistente de conversación" o simplemente "yo". ¿En qué puedo ayudarte hoy?'

# Cargar los Documentos (RAG)

In [5]:
direc = './PDF/'
loader=PyPDFDirectoryLoader(direc)

In [6]:
pdfs = loader.load() # Aquí están todos los PDF

## ETL

- Remplazo de `\n , \n*, \n\n*` por ` " " ` y las palabras que empiezan con `\nlimpieza` solo quedarían como "limpieza" 

In [ ]:
for pdf in pdfs:
    pdf.page_content = re.sub(r'\n\*?\s*\n\*?', ' ', pdf.page_content)
    pdf.page_content = re.sub(r'\s+\b', ' ', pdf.page_content) 
pdfs
    

# Splitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

### Hacer los Chunks

In [9]:
chunks=text_splitter.split_documents(pdfs)

In [ ]:
chunks

In [11]:
chunks[0]

Document(metadata={'source': 'PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 0}, page_content='Universidad Católica del Norte REGLAMENTO GENERAL DE DOCENCIA DE PREGRADO JUNIO 2024')

In [12]:
chunks[1]

Document(metadata={'source': 'PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 1}, page_content='ÍNDICE TÍTULO | Disposiciones generale TÍTULO II De la calidad de estudiante TÍTULO IIl Admisión y Matricula TÍTULO IV Estructura y Organización Curriculal ... 7 TÍTULO V Sistema de Créditos Transferibles (SCT) TÍTULO VI Inscripción y renuncia de actividades curriculare TÍTULO VIl Evaluaciones y calificaciones TÍTULO Vill Asignaturas por Tutoría y EXAMEN ..o 18 TÍTULO IX Interrupción de estudios s 19 TÍTULO X Eliminación y Renuncia TÍTULO XI Cambios de carrera o de programa y cambios de sede. TÍTULO XIl Traslado de Universidad TÍTULO XIl Movilidad estudiantil e 27 TÍTULO XV Egreso y Titulación ...t 30 Artículo transitorio ...O aA 31')

# Hacer el embedding

## Obtener el modelo de embedding desde HuggingFace

In [13]:
embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


# Chroma


### Creación de la base de datos

- En esta etapa se crear el embbeding de los chunks gracias al modelo, se crear un directorio donde se almacenarán los embbeding y se crea la colección

In [14]:
vs = Chroma.from_documents(
    documents=chunks,
    embedding=embed_model,
    persist_directory="chroma_db_dir",  # Local mode with in-memory storage only
    collection_name="reglamentos_UCN"
)

- Se crea la instancia de la bd, enlazando todo, el modelo de embedding, el directorio donde está la bd y cual colección se deben conectar.

In [15]:
vectorstore = Chroma(embedding_function=embed_model,
                     persist_directory="./chroma_db_dir/",
                     collection_name="reglamentos_UCN")

- Se crea un objeto de tipo retriever a partir de la instancia de Chroma: Los retrievers son componentes que permiten buscar y recuperar información específica de la base de datos de embeddings. Proporcionan una interfaz para interactuar con los datos almacenados.

In [61]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={'k': 3}) 
#Hay arte en elgir el tipo de search_type y la cantidad de k

### Hacer el Template Del Promt

In [62]:
custom_prompt_template = """ 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: {context}
Pregunta: {question}

responde siempre en español a no ser que te pidan lo contrario
Respuesta útil:

"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

-  Este método crea una instancia de un sistema de preguntas y respuestas que integra recuperación de información y un modelo de lenguaje.

In [63]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs={"prompt": prompt,"verbose":True})

## Prompts De prueba

In [64]:
def full_response(response):
    best_chunk=response['source_documents'][0]

    file_name = best_chunk.metadata['source'].split('\\')[-1].split('/')[-1]

    page_number = int(best_chunk.metadata['page']) + 1

    full_response =response['result'] + f'\nLos detalles de esta respuesta fueron recuperados del documento "{file_name}", en la página número {page_number}.'
    print(full_response)

### Reglamento-General-de-Docencia-de-Pregrado-2024-2

In [65]:
response = qa.invoke({"query": "¿Que es un estudiante regular?"})
full_response(response)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: Art. 6 Estudiante regular: Es quien ingresa a través de los procedimientos de admisión, que establece el Título II| del presente Reglamento, y adscrito a un programa o carrera determinada, cursa estudios conduce

In [66]:
response = qa.invoke({"query": "¿Que es un estudiante egresado/a?"})
full_response(response)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: Art. 9. Estudiante especial: Estudiante que se inscribe, con la — autorización correspondiente de la unidad académica respectiva, en determinadas actividades académicas para adquirir o desarrollar competencias. 

In [67]:
response = qa.invoke({"query": "¿Que es una actividad curricular?"})
full_response(response)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: competencias y exigencias, así como el año en que éstas hubiesen sido aprobadas. Estudiantes que realicen actividades extracurriculares que se encuentren alineadas con el PEl, podrán solicitar la convalidación d

In [68]:
response = qa.invoke({"query": "¿Cuales son los tipos de estudiantes de intercambios hay?"})
response



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: La resolución de esta solicitud esta sujeta a la ponderación de diversos criterios tales como puntaje de admisión, número de cupos disponibles, test vocacional y notas obtenidas en la carrera de origen, sin perj

{'query': '¿Cuales son los tipos de estudiantes de intercambios hay?',
 'result': 'Según el texto, no hay una sección específica dedicada a "estudiantes de intercambio" o "tipos de estudiantes de intercambio". Sin embargo, hay un artículo (Art. 56) que habla sobre "traslado de universidad", que puede ser relevante para tu pregunta.\n\nSegún Art. 56, se entenderá por traslado de universidad al acto en virtud del cual un/a estudiante se cambia de otra universidad acreditada a la Universidad Católica del Norte, con el objeto de proseguir la misma u otra carrera o programa. Esto sugiere que hay estudiantes que pueden estar haciendo intercambios entre universidades.\n\nNo obstante, no hay una clasificación específica de "tipos de estudiantes de intercambio" en el texto proporcionado.',
 'source_documents': [Document(metadata={'page': 25, 'source': 'PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf'}, page_content='La resolución de esta solicitud esta sujeta a la ponderación de dive

In [69]:
response = qa.invoke({"query": "¿requisito  mínimo  para  presentar  la  solicitud  de  traslado,?"})
full_response(response)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: Art. 57. Se considerará requisito mínimo para presentar la solicitud de traslado, acompañar la siguiente documentación oficial, en original: a) Certificado de alumno/a regular de la universidad de origen. b) Cer

### Reglamento-permanencia-UCN.Nuevo_.-D4-2023-salida

In [73]:
response = qa.invoke({"query": "¿Que es una Infracción leve?"})
response['result']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: e Tener inscrita al menos 2 asignaturas por semestre en curso y no quedar con menos de 12 SCT luego de la renuncia. 
* Tener inscrita en primera o segunda oportunidad aquella(s) actividad(es) a la(s) cual(es) de

'Según el artículo 6 del texto proporcionado, se considerarán infracciones menos graves las siguientes conductas:\n\na) La reincidencia en más de una conducta calificada como infracción leve o la recurrencia de una de ellas en más de un oportunidad.\n\nb) Resistirse, de cualquier forma, a cumplir las disposiciones emanadas de las autoridades de la Universidad, Unidad Académico/ás y personal de apoyo a la academia que se encuentren respaldas en la reglamentación UCN.\n\nc) La utilización del nombre o logotipo de la Universidad o sus autoridades, sin previa autorización del funcionario competente o el mal uso de estas.\n\nd) El arrogarse indebidamente la representación de algún(s) miembros de la comunidad universitaria.\n\nEs importante destacar que no se encuentra definición explícita de "infracción leve" en el texto proporcionado, pero si se menciona que las conductas mencionadas anteriormente se consideran infracciones menos graves.'

In [75]:
response = qa.invoke({"query": "¿Cuales son los tipos de infracciones que existen?"})
response['result']



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
- Trata de no responder con "Se puede inferir", solo responde con respusetas específicas con respecto al texto

Contexto: en horarioy/o díainhábil, no habiéndolo consentido por escritoelo lafuncionario/a paraeldesarrollo deunadeterminada actividad. Artículo6 Seconsiderarán infracciones menosgraves: a) Laincurrencia enmásdeunaconduc

'Según el artículo 6 del Reglamento Interno de la Universidad Católica del Norte, Campus Guayacán, hay varios tipos de infracciones. \n\nSe considerarán infracciones menos graves:\n\na) La recurrencia en más de una ocasión de una conducta calificada como infracción leve o la recurrencia de dicha infracción en más de un oportunidad.\nb) Resistirse, en cualquier forma, a cumplir las disposiciones emanadas de las autoridades de la Universidad, Unidad Académico/as y personal de apoyo a la academia que se encuentren respaldas en el reglamento UCN.\nc) La utilización del nombre o logotipo de la Universidad o de sus autoridades, sin previa autorización del funcionario competente o el mal uso de estas.\nd) El arrogarse indebidamente la representación de alguno(s) miembros de la comunidad universitaria.\n\ne) Tener inscrita al menos 2 asignaturas por semestre en curso y no quedar con menos de 12 SCT luego de la renuncia. \n* Tener inscrita en primera o segunda oportunidad aquella(s) actividad(e